## Final Project Submission

Please fill out:
* Student name: Nastaran Nazemian
* Student pace: part time
* Scheduled project review date/time: ??????????
* Instructor name: Eli Thomas
* Blog post URL:?????????????


In [1]:
from bs4 import BeautifulSoup
import requests 
from datetime import datetime
from datetime import date
import json
import time

In [2]:
def search_movies_page(index):
    html_page = requests.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2019-01-01,2020-01-01&countries=us&start=' + str(index * 50 + 1))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    movies = soup.find_all('div', "lister-item-content")
    urls = []
    for movie in movies:
        movie_url = 'https://www.imdb.com' + movie.find('h3', 'lister-item-header').find('a')['href'] 
#         print(movie_url)
        urls.append(movie_url)
    return(urls)

# url = search_movies_page(0)
#url

In [3]:
def movie_page(url):
    movie_page = requests.get(url)    
    return BeautifulSoup(movie_page.content, 'html.parser')

# this_movie_page = movie_page("https://www.imdb.com/title/tt8367814/")
#movie_page(url)

In [4]:
#def movie_id(index,movie_index)    
#     title = {}
#     movie_title_id =search_movies_page(index)[movie_index].split('/')
#     title_id = movie_title_id[-2]
#     title[Title] = title_id 
#return(title)

In [5]:
def movie_id(url):
    title = {}
    movie_title_id = url.split('/')
    title['Title_id'] = movie_title_id[-2]
    return title
#movie_id(url)

In [6]:
def boxoffice(this_movie_page):
    box_office = ['Budget','Gross USA','Cumulative Worldwide Gross','Opening Weekend USA']
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    for x in box_office:
        try:
            finance_value = this_movie_page.find('h4', string= x + ':')
            results[x] = finance_value.parent.contents[2].replace('$',"").replace(',',"").strip()
        except:
            results[x] = None
    return results
# boxoffice(this_movie_page)

In [7]:
def movie_info(this_movie_page):
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    try:
        results['Title'] = this_movie_page.find('div', class_="title_wrapper").find('h1', class_="").contents[0].replace('\xa0','')
    except:
        results['Title'] = None
    try:
        results['Year'] = this_movie_page.find('div', class_="title_wrapper").find('a').contents[0]  
    except:
        results['Year'] = None
    try:
        results['Popularity'] = this_movie_page.find_all('div', class_ = "titleReviewBarSubItem")[-1].find('span', "subText").contents[0].strip().split(' ')[0].strip().replace(',','')
    except:
        results['Popularity'] = None
    return results
# movie_info(this_movie_page)

In [8]:
def movie_genres(this_movie_page):
    #this_movie_page = movie_page(index, movie_index)
    genres_out = {}
    genres_list = this_movie_page.find('h4', string = 'Genres:').parent.text.replace('|','').split()
    genres_out['Genre'] = genres_list[1:]
    return genres_out
# genres_out = movie_genres(this_movie_page)
# print(genres_out['Genre'] )

In [9]:
def movie_runtime(this_movie_page):
    #this_movie_page = movie_page(index,movie_index)
    runtime_out = {}
    runtime = this_movie_page.find('h4',string="Runtime:").find_next_sibling('time').text.split()
    runtime_out['Runtime_min'] = runtime[0]
    return runtime_out
#movie_runtime(url)

In [10]:
def movie_release_day(this_movie_page):
    #this_movie_page = movie_page(index,movie_index)
    release_day = {}
    release_day_list = this_movie_page.find('div', class_ = "subtext").find('a', title = "See more release dates").text.split()
    date = " ".join(release_day_list[0:3])
    release_day['Release_day'] = datetime.strptime(date, '%d %B %Y').strftime("%m/%d/%Y")
    return release_day
#movie_release_day(url)

In [11]:
def rating (this_movie_page):
    results = {}
    #this_movie_page = movie_page(index, movie_index)
    try:
        check_rating = this_movie_page.find('div', class_="ratingValue")
        rating = check_rating.find('strong')['title'].split()
        results['IMDB_Rate'] = rating[0]
        results['Rating_count'] = rating[3].replace(',','')
    except:
        results['IMDB_Rate'] = None
        results['Rating_count'] = None
        
    try:  
        check_value = this_movie_page.find('div',"metacriticScore score_mixed titleReviewBarSubItem")
        results['Meta_Score'] = check_value.find('span').text
    except:
        results['Meta_Score'] = None
    return results
#rating(url)

In [12]:
def casts_names(this_movie_page):
    roles = [('Director',2),('Writer',2), ('Star',3)]
    results = {}

            #this_movie_page = movie_page(index,movie_index)
    for role in roles:
        cast_role = (role[0]+':')

        name_list = []
        try:
            check_role = this_movie_page.find('h4',string=role[0]+':')
            if (check_role is None):
                check_role = this_movie_page.find('h4',string=role[0]+'s:')
            names = check_role.find_next_siblings('a')
            for name in names[:role[1]]:
                name_list.append(name.text)
            results[role[0]] = name_list
        except:
            results[role[0]] = None
    return results
# casts_names(this_movie_page)

In [13]:
##Find Number_User_Review, Number_Critical_Review
def review (this_movie_page):
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    check_value = this_movie_page.find('div', "titleReviewBarItem titleReviewbarItemBorder")
    if check_value is not None:
        review = check_value.find('span', "subText")
        user = review.find('a').text.split()
        results['User_Review_Number'] = user[0].replace(',','')
        critic = review.find('a', href = "externalreviews").text.split()
        results['Critical_Review_Number'] = critic[0]
    else:
        results['User_Review_Number'] = None
        results['Critical_Review_Number'] = None
    return(results)
#review(url)

In [14]:
##Find Meta_rating,  Genera, and Release_Day---->sub_text(Tird part)
def meta_rate (this_movie_page):
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    check_value = this_movie_page.find('div', class_="title_wrapper")
    if check_value is not None:
        sub_text = check_value.find('div', class_ = "subtext").text.split('|')
        meta_rating = sub_text[0].strip()
        results['Meta_Rating'] = meta_rating
    else:
        results['Meta_Rating'] = None
    
    return(results)

#meta_rate(url)

In [15]:
# this_movie_page = movie_page(url)
def one_movie_info(url):
    this_movie_page = movie_page(url)
    movie_data = {}
#     movie_function_list = [movie_id(url), 
#                            boxoffice(this_movie_page),
#                            movie_info(this_movie_page),
#                            movie_genres(this_movie_page),
#                            movie_runtime(this_movie_page),
#                            movie_release_day(this_movie_page),
#                            rating (this_movie_page),
#                            casts_names(this_movie_page),
#                            review (this_movie_page),
#                            meta_rate (this_movie_page)]
#     for function in movie_function_list:
#         movie_data.update(function)
        
    
    try:
        movie_data.update(movie_id(url))
    except:
        print ("ERROR:", url)
    try:
        movie_data.update(boxoffice(this_movie_page))
    except:
        print ("ERROR:", url)
    try:
        movie_data.update(movie_info(this_movie_page))
    except:
        print ("ERROR:", url)
    try:
        movie_data.update(movie_genres(this_movie_page))
    except:
        print ("ERROR:", url)
    try:
        movie_data.update(movie_runtime(this_movie_page))
    except:
        print ("ERROR:", url)
    try:
        movie_data.update(movie_release_day(this_movie_page))
    except:
        print ("ERROR:", url)
    try:
        movie_data.update(rating (this_movie_page))
    except:
        print ("ERROR:", url)
    try:
        movie_data.update(casts_names(this_movie_page))
    except:
        print ("ERROR:", url)
    try:
        movie_data.update(review (this_movie_page))
    except:
        print ("ERROR:", url)
    try:
        movie_data.update(meta_rate (this_movie_page))
    except:
        print ("ERROR:", url)
    
    return(movie_data)
# len(one_movie_info(url))  


In [16]:
# urls = search_movies_page(0)
# #print(len(urls))
# movies_data = []
# for url in urls:
# #     this_movie_page = movie_page(url)
#     one_row = one_movie_info(url)
#     movies_data.append(one_row)
#     #print(len(movies_data))
# print(len(movies_data))

In [21]:
movies_data = []
for index in range (0, 69):
    urls = search_movies_page(index)
#     print(len(urls))
    for url in urls:
        print(url)
#         time.sleep(1)
        one_row = one_movie_info(url)
        movies_data.append(one_row)
#         print(len(movies_data))
print('end', len(movies_data))
with open('data_0422_0503PM.json', 'w') as outfile:
    json.dump(movies_data, outfile)

https://www.imdb.com/title/tt8367814/
https://www.imdb.com/title/tt7131622/
https://www.imdb.com/title/tt8946378/
https://www.imdb.com/title/tt8579674/
https://www.imdb.com/title/tt3281548/
https://www.imdb.com/title/tt2527338/
https://www.imdb.com/title/tt6673612/
https://www.imdb.com/title/tt4154796/
https://www.imdb.com/title/tt7286456/
https://www.imdb.com/title/tt7975244/
https://www.imdb.com/title/tt6189022/
https://www.imdb.com/title/tt2584384/
https://www.imdb.com/title/tt8772262/
https://www.imdb.com/title/tt8079248/
https://www.imdb.com/title/tt5727208/
https://www.imdb.com/title/tt7349950/
https://www.imdb.com/title/tt1950186/
https://www.imdb.com/title/tt2066051/
https://www.imdb.com/title/tt1302006/
https://www.imdb.com/title/tt6320628/
https://www.imdb.com/title/tt6105098/
https://www.imdb.com/title/tt4520988/
https://www.imdb.com/title/tt5697572/
https://www.imdb.com/title/tt7984734/
https://www.imdb.com/title/tt6394270/
https://www.imdb.com/title/tt8058874/
https://www.

https://www.imdb.com/title/tt5974030/
https://www.imdb.com/title/tt8155288/
https://www.imdb.com/title/tt3584496/
https://www.imdb.com/title/tt8075260/
https://www.imdb.com/title/tt5598292/
https://www.imdb.com/title/tt7616148/
https://www.imdb.com/title/tt3256226/
https://www.imdb.com/title/tt6063090/
https://www.imdb.com/title/tt8745960/
https://www.imdb.com/title/tt7832848/
https://www.imdb.com/title/tt4797182/
https://www.imdb.com/title/tt5792656/
https://www.imdb.com/title/tt5363618/
https://www.imdb.com/title/tt8484012/
https://www.imdb.com/title/tt8085790/
https://www.imdb.com/title/tt6838918/
https://www.imdb.com/title/tt4353250/
https://www.imdb.com/title/tt8186318/
https://www.imdb.com/title/tt7390588/
https://www.imdb.com/title/tt10008784/
https://www.imdb.com/title/tt9419834/
https://www.imdb.com/title/tt4585910/
ERROR: https://www.imdb.com/title/tt4585910/
https://www.imdb.com/title/tt5125894/
https://www.imdb.com/title/tt6837832/
ERROR: https://www.imdb.com/title/tt683783

https://www.imdb.com/title/tt4669252/
https://www.imdb.com/title/tt4328094/
https://www.imdb.com/title/tt7476524/
https://www.imdb.com/title/tt8750122/
https://www.imdb.com/title/tt9534808/
https://www.imdb.com/title/tt7434324/
https://www.imdb.com/title/tt10521814/
https://www.imdb.com/title/tt6544220/
https://www.imdb.com/title/tt7781432/
ERROR: https://www.imdb.com/title/tt7781432/
https://www.imdb.com/title/tt8589210/
ERROR: https://www.imdb.com/title/tt8589210/
https://www.imdb.com/title/tt5848416/
https://www.imdb.com/title/tt9612982/
https://www.imdb.com/title/tt8009906/
https://www.imdb.com/title/tt9358160/
https://www.imdb.com/title/tt7167686/
https://www.imdb.com/title/tt8396294/
https://www.imdb.com/title/tt6910006/
https://www.imdb.com/title/tt9614440/
https://www.imdb.com/title/tt9318280/
https://www.imdb.com/title/tt8890158/
ERROR: https://www.imdb.com/title/tt8890158/
https://www.imdb.com/title/tt9177040/
https://www.imdb.com/title/tt10621180/
ERROR: https://www.imdb.com

https://www.imdb.com/title/tt8169994/
ERROR: https://www.imdb.com/title/tt8169994/
https://www.imdb.com/title/tt10086502/
https://www.imdb.com/title/tt3699702/
https://www.imdb.com/title/tt7643622/
https://www.imdb.com/title/tt8354582/
https://www.imdb.com/title/tt7403614/
https://www.imdb.com/title/tt7772498/
https://www.imdb.com/title/tt8160864/
https://www.imdb.com/title/tt9238392/
ERROR: https://www.imdb.com/title/tt9238392/
https://www.imdb.com/title/tt9119200/
ERROR: https://www.imdb.com/title/tt9119200/
https://www.imdb.com/title/tt8964214/
https://www.imdb.com/title/tt10510654/
https://www.imdb.com/title/tt4205896/
ERROR: https://www.imdb.com/title/tt4205896/
https://www.imdb.com/title/tt5580688/
ERROR: https://www.imdb.com/title/tt5580688/
https://www.imdb.com/title/tt9469842/
https://www.imdb.com/title/tt6345894/
ERROR: https://www.imdb.com/title/tt6345894/
https://www.imdb.com/title/tt8184766/
https://www.imdb.com/title/tt7262384/
ERROR: https://www.imdb.com/title/tt7262384/

https://www.imdb.com/title/tt10149756/
https://www.imdb.com/title/tt7677420/
https://www.imdb.com/title/tt5258074/
https://www.imdb.com/title/tt7120400/
ERROR: https://www.imdb.com/title/tt7120400/
https://www.imdb.com/title/tt7144186/
https://www.imdb.com/title/tt7144200/
https://www.imdb.com/title/tt10461980/
https://www.imdb.com/title/tt10071352/
ERROR: https://www.imdb.com/title/tt10071352/
https://www.imdb.com/title/tt5183990/
ERROR: https://www.imdb.com/title/tt5183990/
ERROR: https://www.imdb.com/title/tt5183990/
https://www.imdb.com/title/tt8179944/
https://www.imdb.com/title/tt5478746/
https://www.imdb.com/title/tt9086910/
https://www.imdb.com/title/tt10370822/
https://www.imdb.com/title/tt8479346/
https://www.imdb.com/title/tt8789416/
ERROR: https://www.imdb.com/title/tt8789416/
https://www.imdb.com/title/tt8394314/
https://www.imdb.com/title/tt10925838/
https://www.imdb.com/title/tt7246106/
https://www.imdb.com/title/tt5265974/
ERROR: https://www.imdb.com/title/tt5265974/
ht

...

ERROR: https://www.imdb.com/title/tt11750238/
https://www.imdb.com/title/tt10821570/
ERROR: https://www.imdb.com/title/tt10821570/
ERROR: https://www.imdb.com/title/tt10821570/
https://www.imdb.com/title/tt9595666/
ERROR: https://www.imdb.com/title/tt9595666/
https://www.imdb.com/title/tt10409102/
ERROR: https://www.imdb.com/title/tt10409102/
https://www.imdb.com/title/tt10928650/
ERROR: https://www.imdb.com/title/tt10928650/
https://www.imdb.com/title/tt10311886/
ERROR: https://www.imdb.com/title/tt10311886/
ERROR: https://www.imdb.com/title/tt10311886/
https://www.imdb.com/title/tt8917252/
ERROR: https://www.imdb.com/title/tt8917252/
https://www.imdb.com/title/tt10092368/
ERROR: https://www.imdb.com/title/tt10092368/
https://www.imdb.com/title/tt11693328/
ERROR: https://www.imdb.com/title/tt11693328/
https://www.imdb.com/title/tt10285114/
ERROR: https://www.imdb.com/title/tt10285114/
https://www.imdb.com/title/tt9817040/
ERROR: https://www.imdb.com/title/tt9817040/
ERROR: https://www

ERROR: https://www.imdb.com/title/tt8702994/
https://www.imdb.com/title/tt10188198/
https://www.imdb.com/title/tt9819024/
https://www.imdb.com/title/tt9078576/
https://www.imdb.com/title/tt10545182/
ERROR: https://www.imdb.com/title/tt10545182/
ERROR: https://www.imdb.com/title/tt10545182/
https://www.imdb.com/title/tt11009214/
ERROR: https://www.imdb.com/title/tt11009214/
ERROR: https://www.imdb.com/title/tt11009214/
ERROR: https://www.imdb.com/title/tt11009214/
https://www.imdb.com/title/tt11214052/
https://www.imdb.com/title/tt10973926/
ERROR: https://www.imdb.com/title/tt10973926/
https://www.imdb.com/title/tt10347770/
https://www.imdb.com/title/tt11171312/
https://www.imdb.com/title/tt9071828/
ERROR: https://www.imdb.com/title/tt9071828/
https://www.imdb.com/title/tt11547692/
ERROR: https://www.imdb.com/title/tt11547692/
https://www.imdb.com/title/tt9004636/
ERROR: https://www.imdb.com/title/tt9004636/
https://www.imdb.com/title/tt8333758/
ERROR: https://www.imdb.com/title/tt83337

ERROR: https://www.imdb.com/title/tt8449562/
ERROR: https://www.imdb.com/title/tt8449562/
https://www.imdb.com/title/tt9397380/
ERROR: https://www.imdb.com/title/tt9397380/
https://www.imdb.com/title/tt2763278/
ERROR: https://www.imdb.com/title/tt2763278/
https://www.imdb.com/title/tt10732690/
ERROR: https://www.imdb.com/title/tt10732690/
https://www.imdb.com/title/tt9293562/
ERROR: https://www.imdb.com/title/tt9293562/
https://www.imdb.com/title/tt8596970/
https://www.imdb.com/title/tt10654458/
ERROR: https://www.imdb.com/title/tt10654458/
ERROR: https://www.imdb.com/title/tt10654458/
ERROR: https://www.imdb.com/title/tt10654458/
https://www.imdb.com/title/tt11416992/
ERROR: https://www.imdb.com/title/tt11416992/
https://www.imdb.com/title/tt6328708/
ERROR: https://www.imdb.com/title/tt6328708/
https://www.imdb.com/title/tt10426564/
ERROR: https://www.imdb.com/title/tt10426564/
https://www.imdb.com/title/tt11166960/
ERROR: https://www.imdb.com/title/tt11166960/
https://www.imdb.com/ti

ERROR: https://www.imdb.com/title/tt9707108/
https://www.imdb.com/title/tt10155386/
ERROR: https://www.imdb.com/title/tt10155386/
https://www.imdb.com/title/tt8975212/
https://www.imdb.com/title/tt10408950/
ERROR: https://www.imdb.com/title/tt10408950/
https://www.imdb.com/title/tt11089638/
https://www.imdb.com/title/tt10399072/
ERROR: https://www.imdb.com/title/tt10399072/
https://www.imdb.com/title/tt9579746/
https://www.imdb.com/title/tt9859056/
ERROR: https://www.imdb.com/title/tt9859056/
https://www.imdb.com/title/tt11403000/
ERROR: https://www.imdb.com/title/tt11403000/
https://www.imdb.com/title/tt11760864/
ERROR: https://www.imdb.com/title/tt11760864/
https://www.imdb.com/title/tt11110366/
ERROR: https://www.imdb.com/title/tt11110366/
ERROR: https://www.imdb.com/title/tt11110366/
https://www.imdb.com/title/tt11760676/
ERROR: https://www.imdb.com/title/tt11760676/
https://www.imdb.com/title/tt11310802/
https://www.imdb.com/title/tt8929554/
ERROR: https://www.imdb.com/title/tt892

https://www.imdb.com/title/tt12064020/
https://www.imdb.com/title/tt11377744/
https://www.imdb.com/title/tt12126242/
ERROR: https://www.imdb.com/title/tt12126242/
https://www.imdb.com/title/tt8260936/
https://www.imdb.com/title/tt12145554/
ERROR: https://www.imdb.com/title/tt12145554/
https://www.imdb.com/title/tt12162552/
https://www.imdb.com/title/tt12175974/
ERROR: https://www.imdb.com/title/tt12175974/
https://www.imdb.com/title/tt12179880/
https://www.imdb.com/title/tt12180402/
https://www.imdb.com/title/tt12181440/
ERROR: https://www.imdb.com/title/tt12181440/
https://www.imdb.com/title/tt12193310/
ERROR: https://www.imdb.com/title/tt12193310/
end 3413


In [18]:
import re
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
df = pd.DataFrame(movies_data)
df.head()


,Title_id,Budget,Gross USA,Cumulative Worldwide Gross,Opening Weekend USA,Title,Year,Popularity,Genre,Runtime_min,Release_day,IMDB_Rate,Rating_count,Meta_Score,Director,Writer,Star,User_Review_Number,Critical_Review_Number,Meta_Rating
0,tt11934788,None,None,None,None,Democracy Gone Wrong,2019,None,[Drama],87,05/10/2019,None,None,None,[Germany Kent],"[Germany Kent, Eden Rashad]","[Carlos Chatman, Alex DeSantiago, Armondo DeSa...",None,None,1h 27min
1,tt2076961,500,None,None,None,The Untelling,2019,None,[Horror],9,04/25/2020,None,None,None,[Russell Whaley],[Russell Whaley],"[Jessica Felice, Judith Grannas, John Livesay]",None,None,9min
2,tt12064020,78000,None,None,None,Friday Afternoon,2019,None,[Drama],77,08/02/2019,None,None,None,[Paige Campbell],[Paige Campbell],"[Becca Ballenger, Kim Blanck, Andrew R. Butler]",None,None,1h 17min
3,tt11377744,15000,None,None,None,Captain Franklin: Infinity Fork,2019,None,[Sci-Fi],46,09/27/2019,None,None,None,"[Porter Edwards, Bryson Sams]","[Porter Edwards, Bryson Sams]","[Matthew Dyra, Tucker Vaughn, Hugh Inman]",None,None,46min
4,tt12126242,20000,None,None,None,Fatal Encounter Rise and Fight,2019,None,[Action],NaN,05/14/2019,None,None,None,[Patreck Farley],"[Patreck Farley, Richard Hyer]","[Michael Barnes, Patreck Farley, Steven Hirsch]",None,None,Action


In [22]:
df.describe()

TypeError: unhashable type: 'list'

In [ ]:
df.info()
df.shape
